# Major Model Analysis Ahead

In [43]:
from utils.model_analysis_local import * 
import os
import matplotlib.pyplot as plt
from nilearn import datasets, plotting
from PIL import Image
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')
import regex as re
import pandas as pd

# Define Model, Subject and Image for Analysis

In [ ]:
#Change as needed
checkpoint_path = '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_200_EPOCHS100_BATCHSIZE_16_TIME_2023-12-07_23:18:19.pt'
backbone = 'alexnet'
subject_id = 1
image_id = 0

# Loading and (Pre-)Processing

In [ ]:
#load images and original activations 
brain, image_paths = load_subject_data(subject_id)

#pick image and activation
images = []
for i in range(len(image_paths)):
    image = np.load(image_paths[i])
    image = preprocess(image)
    images.append(image)
activation = torch.Tensor(brain)

In [ ]:
output_size = 100

# Check if GPU is available and if not, use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Load in model & Create feature extractor
trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
feature_extractor = torch.hub.load('utils', backbone, source = 'local')

trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
trained_model_backbone = trained_model.feature_extractor

if backbone == "alexnet":
    output_layer_names = get_module_names(trained_model_backbone)[1:-1]
else:
    output_layer_names = get_block_names(trained_model_backbone)[:-1]

feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

#Extract features & predict fMRI data 
outputs = feature_extractor(image.unsqueeze(0))
flat_outputs = flatten_features(outputs)
predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
print(predictions)

#Inverse transform of preds with frozen PCA models
pca = get_pca_model(subject_id)
inversed_predictions = predictions.copy()
for key in inversed_predictions:
    #convert prediction tensors to np arrays to make it compatible for inverse pca
    preds = inversed_predictions[key].detach().numpy()
    preds = torch.Tensor(pca.inverse_transform(preds))
    # inverse-pca and store in new dict
    inversed_predictions[key] = preds

# Model Analysis - Whole Model

## Loading and (Pre-)Processing

In [ ]:
#use loaded model from beginning and pass image through
preds = np.zeros(activation.shape)
for i, img in enumerate(images):
    pred = trained_model(img.unsqueeze(0))
    inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
    preds[i,:] = inversed_pred

predictions = torch.Tensor(preds)

## Calculate MNNPC

In [ ]:
#for single image
mnnpc = MNNPC()
score = mnnpc(pred = predictions, gt=activation)
score

# Big Ass Loop for all Subjects

In [5]:
#Change as needed
checkpoint_path = '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_200_EPOCHS100_BATCHSIZE_16_TIME_2023-12-07_23:18:19.pt'
backbone = 'alexnet'
subject_ids = list(range(1,8+1))
scores = []

In [6]:
for subject_id in subject_ids:
    #load images and original activations 
    brain, image_paths = load_subject_data(subject_id)

    #pick image and activation
    images = []
    for i in range(len(image_paths)):
        image = np.load(image_paths[i])
        image = preprocess(image)
        images.append(image)
    activation = torch.Tensor(brain)


    output_size = 100

    # Check if GPU is available and if not, use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #Load in model & Create feature extractor
    trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
    feature_extractor = torch.hub.load('utils', backbone, source = 'local')

    trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
    trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
    trained_model_backbone = trained_model.feature_extractor

    if backbone == "alexnet":
        output_layer_names = get_module_names(trained_model_backbone)[1:-1]
    else:
        output_layer_names = get_block_names(trained_model_backbone)[:-1]

    feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

    #Extract features & predict fMRI data 
    outputs = feature_extractor(image.unsqueeze(0))
    flat_outputs = flatten_features(outputs)
    predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
    print(predictions)

    #Inverse transform of preds with frozen PCA models
    pca = get_pca_model(subject_id)
    inversed_predictions = predictions.copy()
    for key in inversed_predictions:
        #convert prediction tensors to np arrays to make it compatible for inverse pca
        preds = inversed_predictions[key].detach().numpy()
        preds = torch.Tensor(pca.inverse_transform(preds))
        # inverse-pca and store in new dict
        inversed_predictions[key] = preds

    #use loaded model from beginning and pass image through
    preds = np.zeros(activation.shape)
    for i, img in enumerate(images):
        pred = trained_model(img.unsqueeze(0))
        inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
        preds[i,:] = inversed_pred

    predictions = torch.Tensor(preds)

    #calculate MNNPC score
    mnnpc = MNNPC()
    score = mnnpc(pred = predictions, gt=activation)

    #append to list
    scores.append(score)


print('final MNNPC score: ', np.mean(scores))  

Shape of pca_brain:  (984, 39548)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_200_EPOCHS100_BATCHSIZE_16_TIME_2023-12-07_23:18:19.pt'

In [ ]:
scores

# Even Bigger loop over multiple models and all subjects

In [20]:
comparison = {}
models = []
mnnpc_scores = []

## Alexnet Models

In [18]:
#Change as needed
paths = [#'/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.0003_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_13:59:11.pt',
        '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2023-12-14_16:29:15.pt',
        #'/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_13:10:42.pt'
         ]
                   
backbone = 'alexnet'
subject_ids = list(range(1,8+1))

In [21]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)
    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        scores = []
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)


        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        if backbone == "alexnet":
            output_layer_names = get_module_names(trained_model_backbone)[1:-1]
        else:
            output_layer_names = get_block_names(trained_model_backbone)[:-1]

        feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

        #Extract features & predict fMRI data 
        outputs = feature_extractor(image.unsqueeze(0))
        flat_outputs = flatten_features(outputs)
        predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
        print(predictions)

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        inversed_predictions = predictions.copy()
        for key in inversed_predictions:
            #convert prediction tensors to np arrays to make it compatible for inverse pca
            preds = inversed_predictions[key].detach().numpy()
            preds = torch.Tensor(pca.inverse_transform(preds))
            # inverse-pca and store in new dict
            inversed_predictions[key] = preds

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)

        #calculate MNNPC score
        mnnpc = MNNPC()
        score = mnnpc(pred = predictions, gt=activation)

        #append to list
        scores.append(score)
        print(f'Subject {subject_id} done.')

    models.append(model_name)
    mnnpc_scores.append(np.mean(scores))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=921

RuntimeError: Error(s) in loading state_dict for ResNet1HeadID:
	Missing key(s) in state_dict: "shared.sequntial_model.0.weight", "shared.sequntial_model.0.bias", "shared.sequntial_model.1.weight", "shared.sequntial_model.1.bias", "shared.sequntial_model.2.weight", "shared.sequntial_model.2.bias", "shared.sequntial_model.3.weight", "shared.sequntial_model.3.bias", "shared.sequntial_model.3.running_mean", "shared.sequntial_model.3.running_var", "shared.sequntial_model.5.weight", "shared.sequntial_model.5.bias", "shared.sequntial_model.6.weight", "shared.sequntial_model.6.bias", "shared.sequntial_model.7.weight", "shared.sequntial_model.7.bias", "shared.sequntial_model.8.weight", "shared.sequntial_model.8.bias", "shared.sequntial_model.8.running_mean", "shared.sequntial_model.8.running_var", "shared.sequntial_model.10.weight", "shared.sequntial_model.10.bias", "shared.sequntial_model.11.weight", "shared.sequntial_model.11.bias", "shared.sequntial_model.12.weight", "shared.sequntial_model

## Alexnet with Simple Head

In [26]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.0003_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_13:59:11.pt',
        #'/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2023-12-14_16:29:15.pt',
        '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_13:10:42.pt'
         ]
                   
backbone = 'alexnet'
subject_ids = list(range(1,8+1))

In [27]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + '_SIMPLE_HEAD'
    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        scores = []
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)


        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=True).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        if backbone == "alexnet":
            output_layer_names = get_module_names(trained_model_backbone)[1:-1]
        else:
            output_layer_names = get_block_names(trained_model_backbone)[:-1]

        feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

        #Extract features & predict fMRI data 
        outputs = feature_extractor(image.unsqueeze(0))
        flat_outputs = flatten_features(outputs)
        predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
        print(predictions)

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        inversed_predictions = predictions.copy()
        for key in inversed_predictions:
            #convert prediction tensors to np arrays to make it compatible for inverse pca
            preds = inversed_predictions[key].detach().numpy()
            preds = torch.Tensor(pca.inverse_transform(preds))
            # inverse-pca and store in new dict
            inversed_predictions[key] = preds

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)

        #calculate MNNPC score
        mnnpc = MNNPC()
        score = mnnpc(pred = predictions, gt=activation)

        #append to list
        scores.append(score)
        print(f'Subject {subject_id} done.')

    models.append(model_name)
    mnnpc_scores.append(np.mean(scores))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## Resnet 18

In [23]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/resnet18_LR0.00015_SAMPLES_all_EPOCHS50_BATCHSIZE_128_TIME_2023-12-05_02:49:33.pt',
         '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/resnet18_LR1e-05_SAMPLES_all_EPOCHS20_BATCHSIZE_32_TIME_2023-12-06_01:46:24.pt']
                   
backbone = 'resnet18'
subject_ids = list(range(1,8+1))

In [24]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)

    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        scores = []
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)


        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        if backbone == "alexnet":
            output_layer_names = get_module_names(trained_model_backbone)[1:-1]
        else:
            output_layer_names = get_block_names(trained_model_backbone)[:-1]

        feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

        #Extract features & predict fMRI data 
        outputs = feature_extractor(image.unsqueeze(0))
        flat_outputs = flatten_features(outputs)
        predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
        print(predictions)

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        inversed_predictions = predictions.copy()
        for key in inversed_predictions:
            #convert prediction tensors to np arrays to make it compatible for inverse pca
            preds = inversed_predictions[key].detach().numpy()
            preds = torch.Tensor(pca.inverse_transform(preds))
            # inverse-pca and store in new dict
            inversed_predictions[key] = preds

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)

        #calculate MNNPC score
        mnnpc = MNNPC()
        score = mnnpc(pred = predictions, gt=activation)

        #append to list
        scores.append(score)
        print(f'Subject {subject_id} done.')

    models.append(model_name)
    mnnpc_scores.append(np.mean(scores))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Resnet 34

In [28]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/resnet34_LR1e-05_SAMPLES_all_EPOCHS20_BATCHSIZE_32_TIME_2023-12-06_01:30:18.pt']
backbone = 'resnet34'
subject_ids = list(range(1,8+1))

In [29]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)

    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        scores = []
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)


        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        if backbone == "alexnet":
            output_layer_names = get_module_names(trained_model_backbone)[1:-1]
        else:
            output_layer_names = get_block_names(trained_model_backbone)[:-1]

        feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

        #Extract features & predict fMRI data 
        outputs = feature_extractor(image.unsqueeze(0))
        flat_outputs = flatten_features(outputs)
        predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
        print(predictions)

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        inversed_predictions = predictions.copy()
        for key in inversed_predictions:
            #convert prediction tensors to np arrays to make it compatible for inverse pca
            preds = inversed_predictions[key].detach().numpy()
            preds = torch.Tensor(pca.inverse_transform(preds))
            # inverse-pca and store in new dict
            inversed_predictions[key] = preds

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)

        #calculate MNNPC score
        mnnpc = MNNPC()
        score = mnnpc(pred = predictions, gt=activation)

        #append to list
        scores.append(score)
        print(f'Subject {subject_id} done.')

    models.append(model_name)
    mnnpc_scores.append(np.mean(scores))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

## Resnet 101

In [30]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/resnet101_LR0.00015_SAMPLES_all_EPOCHS20_BATCHSIZE_64_TIME_2023-12-05_14:32:18.pt']                   
backbone = 'resnet101'
subject_ids = list(range(1,8+1))

In [31]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1)

    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        scores = []
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)


        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        if backbone == "alexnet":
            output_layer_names = get_module_names(trained_model_backbone)[1:-1]
        else:
            output_layer_names = get_block_names(trained_model_backbone)[:-1]

        feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

        #Extract features & predict fMRI data 
        outputs = feature_extractor(image.unsqueeze(0))
        flat_outputs = flatten_features(outputs)
        predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
        print(predictions)

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        inversed_predictions = predictions.copy()
        for key in inversed_predictions:
            #convert prediction tensors to np arrays to make it compatible for inverse pca
            preds = inversed_predictions[key].detach().numpy()
            preds = torch.Tensor(pca.inverse_transform(preds))
            # inverse-pca and store in new dict
            inversed_predictions[key] = preds

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)

        #calculate MNNPC score
        mnnpc = MNNPC()
        score = mnnpc(pred = predictions, gt=activation)

        #append to list
        scores.append(score)
        print(f'Subject {subject_id} done.')

    models.append(model_name)
    mnnpc_scores.append(np.mean(scores))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 25

## Resnet 152 - Simple Head

In [41]:
#Change as needed
paths = ['/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/resnet152_LR0.0003_SAMPLES_all_EPOCHS20_BATCHSIZE_64_TIME_2023-12-12_19:50:22.pt',
         '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/resnet152_LR0.00015_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_21:03:22.pt'
         ]
                   
backbone = 'resnet152'
subject_ids = list(range(1,8+1))

In [42]:
for checkpoint_path in paths:
    # Use the regex to extract the part after "trained_models/"
    pattern = re.compile(r'_models/(.*)')
    match = pattern.search(checkpoint_path)
    model_name = match.group(1) + 'SIMPLE_HEAD'

    #loop over all subjects to calculate MNNPC per subject
    for subject_id in subject_ids:
        scores = []
        #load images and original activations 
        brain, image_paths = load_subject_data(subject_id)

        #pick image and activation
        images = []
        for i in range(len(image_paths)):
            image = np.load(image_paths[i])
            image = preprocess(image)
            images.append(image)
        activation = torch.Tensor(brain)


        output_size = 100

        # Check if GPU is available and if not, use CPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        #Load in model & Create feature extractor
        trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
        feature_extractor = torch.hub.load('utils', backbone, source = 'local')

        trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor, simple_head=True).eval()
        trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
        trained_model_backbone = trained_model.feature_extractor

        if backbone == "alexnet":
            output_layer_names = get_module_names(trained_model_backbone)[1:-1]
        else:
            output_layer_names = get_block_names(trained_model_backbone)[:-1]

        feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

        #Extract features & predict fMRI data 
        outputs = feature_extractor(image.unsqueeze(0))
        flat_outputs = flatten_features(outputs)
        predictions = make_prediction(trained_model, flat_outputs, trained_model.head.in_features)
        print(predictions)

        #Inverse transform of preds with frozen PCA models
        pca = get_pca_model(subject_id)
        inversed_predictions = predictions.copy()
        for key in inversed_predictions:
            #convert prediction tensors to np arrays to make it compatible for inverse pca
            preds = inversed_predictions[key].detach().numpy()
            preds = torch.Tensor(pca.inverse_transform(preds))
            # inverse-pca and store in new dict
            inversed_predictions[key] = preds

        #use loaded model from beginning and pass image through
        preds = np.zeros(activation.shape)
        for i, img in enumerate(images):
            pred = trained_model(img.unsqueeze(0))
            inversed_pred = pca.inverse_transform(pred.detach().numpy()).flatten()
            preds[i,:] = inversed_pred

        predictions = torch.Tensor(preds)

        #calculate MNNPC score
        mnnpc = MNNPC()
        score = mnnpc(pred = predictions, gt=activation)

        #append to list
        scores.append(score)
        print(f'Subject {subject_id} done.')

    models.append(model_name)
    mnnpc_scores.append(np.mean(scores))
    print(f'Model {model_name} done.')

Shape of pca_brain:  (984, 39548)


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

RuntimeError: Error(s) in loading state_dict for ResNet1HeadID:
	Missing key(s) in state_dict: "shared.weight", "shared.bias", "sub1.weight", "sub1.bias", "sub2.weight", "sub2.bias", "sub3.weight", "sub3.bias", "sub4.weight", "sub4.bias", "sub5.weight", "sub5.bias", "sub6.weight", "sub6.bias", "sub7.weight", "sub7.bias", "sub8.weight", "sub8.bias". 
	Unexpected key(s) in state_dict: "shared.sequntial_model.0.weight", "shared.sequntial_model.0.bias", "shared.sequntial_model.1.weight", "shared.sequntial_model.1.bias", "shared.sequntial_model.2.weight", "shared.sequntial_model.2.bias", "shared.sequntial_model.3.weight", "shared.sequntial_model.3.bias", "shared.sequntial_model.3.running_mean", "shared.sequntial_model.3.running_var", "shared.sequntial_model.3.num_batches_tracked", "shared.sequntial_model.5.weight", "shared.sequntial_model.5.bias", "shared.sequntial_model.6.weight", "shared.sequntial_model.6.bias", "shared.sequntial_model.7.weight", "shared.sequntial_model.7.bias", "shared.s

# Comparing Results

In [49]:
comparison = pd.DataFrame({'Model': models, 'MNNPC Score': mnnpc_scores})
pd.set_option('max_colwidth', 100)
comparison.sort_values('MNNPC Score', ascending=False)

,Model,MNNPC Score
4,alexnet_LR0.00015_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_13:10:42.pt_SIMPLE_HEAD,0.503223
8,resnet152_LR0.00015_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_21:03:22.ptSIMPLE_HEAD,0.502774
7,resnet152_LR0.0003_SAMPLES_all_EPOCHS20_BATCHSIZE_64_TIME_2023-12-12_19:50:22.ptSIMPLE_HEAD,0.502463
0,alexnet_LR0.0003_SAMPLES_all_SIMPLEHEAD_False_EPOCHS50_BATCHSIZE_64_TIME_2023-12-14_16:29:15.pt,0.500000
1,resnet18_LR0.00015_SAMPLES_all_EPOCHS50_BATCHSIZE_128_TIME_2023-12-05_02:49:33.pt,0.499455
6,resnet101_LR0.00015_SAMPLES_all_EPOCHS20_BATCHSIZE_64_TIME_2023-12-05_14:32:18.pt,0.499127
2,resnet18_LR1e-05_SAMPLES_all_EPOCHS20_BATCHSIZE_32_TIME_2023-12-06_01:46:24.pt,0.497231
5,resnet34_LR1e-05_SAMPLES_all_EPOCHS20_BATCHSIZE_32_TIME_2023-12-06_01:30:18.pt,0.495273
3,alexnet_LR0.0003_SAMPLES_all_EPOCHS100_BATCHSIZE_64_TIME_2023-12-11_13:59:11.pt_SIMPLE_HEAD,0.493599


In [50]:
comparison.to_csv('comparison_results.csv')